In [1]:
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition
import seaborn as sns; sns.set_style("whitegrid", {'axes.grid' : False})
import skbio
from scipy.spatial import distance
import random

In [2]:
from matplotlib import rcParams
import string
rcParams['font.family'] = 'Arial'

In [3]:
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [4]:
%matplotlib inline

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
bdr=pd.read_csv('processed_bd_rate.csv',index_col=0)

In [7]:
bdr['tag']=bdr['Salinity'].apply(lambda x: str(x))+'+'+bdr['dis'].apply(lambda x: str(x))

In [8]:
bdr = bdr.iloc[3:,3:].set_index('tag')

In [9]:
bdr.shape

(36, 53)

In [10]:
bdr_standard = pd.DataFrame(StandardScaler().fit_transform(bdr), 
                           index = bdr.index,
                           columns = bdr.columns)

In [11]:
Ar_dist = distance.squareform(distance.pdist(bdr, metric="braycurtis")) # (n x n) distance measure
DM_dist = skbio.stats.distance.DistanceMatrix(Ar_dist, ids=bdr_standard.reset_index().index)

In [12]:
PCoA = skbio.stats.ordination.pcoa(DM_dist,number_of_dimensions=2)

/Users/qiaok/opt/anaconda3/envs/basic_analysis/lib/python3.6/site-packages/skbio/stats/ordination/_principal_coordinate_analysis.py:152: RuntimeWarning: The result contains negative eigenvalues. Please compare their magnitude with the magnitude of some of the largest positive eigenvalues. If the negative ones are smaller, it's probably safe to ignore them, but if they are large in magnitude, the results won't be useful. See the Notes section for more details. The smallest eigenvalue is -0.027313756799077913 and the largest is 0.6611993879280171.
  RuntimeWarning


In [13]:
PCoA.proportion_explained

PC1    0.726284
PC2    0.137943
dtype: float64

In [14]:
result_m = PCoA.samples

In [15]:
result_m.index = bdr.index

In [16]:
result_m

,PC1,PC2
tag,,
0+False,-0.061110,-0.106389
0+False,-0.162591,-0.092015
0+False,-0.011013,-0.107362
0+True,-0.079553,-0.041766
0+True,-0.089961,-0.039914
0+True,0.015884,-0.049365
5+False,-0.147803,-0.005640
5+False,-0.200243,0.000642
5+False,-0.063927,-0.002231


In [27]:
estimator = KMeans(n_clusters=3)
estimator.fit(result_m)
label_pred = estimator.labels_
centroids = estimator.cluster_centers_
inertia = estimator.inertia_